In [75]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import nltk

from datasets import load_dataset
emotions = load_dataset ("go_emotions", "simplified")
depression = load_dataset("ShreyaR/DepressionDetection")
emotions.set_format(type="pandas")
depression.set_format(type="pandas")
df1 = emotions["train"][:]
df2 = depression["train"][:]
df1.drop(columns=['id'], inplace=True) #dropping irrelevant coloums, and also some emotions as this dataset has far too many
df2.rename(columns={'is_depression': 'depression', "clean_text": "text"}, inplace=True)
df2 = df2[df2['depression'] != 0] #removing all rows where depression is 0 as all other emotions data is missing
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df.loc[merged_df['depression'] == 1, 'column2'] = [28]
merged_df.to_csv('combined_data.csv', index=False)
df1.head()

,text,labels
0,My favourite food is anything I didn't have to...,[27]
1,"Now if he does off himself, everyone will thin...",[27]
2,WHY THE FUCK IS BAYLESS ISOING,[2]
3,To make her feel threatened,[14]
4,Dirty Southern Wankers,[3]


In [76]:
debug=load_dataset("emotion")
debug.set_format(type="pandas")
dfd= debug["train"][:]
dfd.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [77]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [78]:
df2.head()

,text,depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1


In [79]:
merged_df.head()

,text,labels,depression
0,My favourite food is anything I didn't have to...,[27],NaN
1,"Now if he does off himself, everyone will thin...",[27],NaN
2,WHY THE FUCK IS BAYLESS ISOING,[2],NaN
3,To make her feel threatened,[14],NaN
4,Dirty Southern Wankers,[3],NaN


In [80]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47241 entries, 0 to 47240
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        47241 non-null  object 
 1   labels      43410 non-null  object 
 2   depression  3831 non-null   float64
dtypes: float64(1), object(2)
memory usage: 1.1+ MB


In [81]:
merged_df['text'] = merged_df['text'].astype(str)
merged_df['author'] = merged_df['author'].astype(str)

KeyError: 'author'

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_valid_df = train_test_split(merged_df, test_size=0.2, random_state=42)
test_df, valid_df = train_test_split(test_valid_df, test_size=0.5, random_state=42)

from datasets import DatasetDict
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)
ds_splits = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})
ds_splits

DatasetDict({
    train: Dataset({
        features: ['text', 'author', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'relief', 'remorse', 'sadness', 'neutral', 'depression', '__index_level_0__'],
        num_rows: 172044
    })
    validation: Dataset({
        features: ['text', 'author', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'relief', 'remorse', 'sadness', 'neutral', 'depression', '__index_level_0__'],
        num_rows: 21506
    })
    test: Dataset({
        features: ['text', 'author', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'lov

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenizer_function(batch):
    return tokenizer(batch["text"], padding= True, truncation= True)

In [ ]:
tokenized_data= ds_splits.map(tokenizer_function,
                             batched= True,
                             batch_size= None)


  0%|          | 0/5378 [12:23<?, ?it/s]

Map: 100%|██████████| 21506/21506 [00:02<00:00, 7607.75 examples/s]


In [ ]:
print(tokenized_data.column_names)

{'train': ['text', 'author', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'relief', 'remorse', 'sadness', 'neutral', 'depression', '__index_level_0__', 'input_ids', 'attention_mask'], 'validation': ['text', 'author', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'relief', 'remorse', 'sadness', 'neutral', 'depression', '__index_level_0__', 'input_ids', 'attention_mask'], 'test': ['text', 'author', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'relief', 'remorse', 'sadness', 'neutral', 'depression', '__index_level_0__', 'input_

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
checkpoint = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = (AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 24).to(device))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds=pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc =accuracy_score(labels, preds)
    return {"accureacy": acc, "f1": f1}

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir= "finetuning-emotion-model",
    num_train_epochs= 2,
    learning_rate= 2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    push_to_hub=False
)

from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics= compute_metrics,
    train_dataset= tokenized_data["train"],
    eval_dataset= tokenized_data["validation"],
    tokenizer= tokenizer
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.